In [6]:
import pandas as pd
import numpy as np
import os
import ast

In [7]:
carpeta_json = 'C:\\Users\\gasty\\Desktop\\PF\\Google Maps\\Reviews-Estado\\review-California'
carpeta_json2 = 'C:\\Users\\gasty\\Desktop\\PF\\Google Maps\Reviews-Estado\\review-Florida'
carpeta_json3 = 'C:\\Users\\gasty\\Desktop\\PF\\Google Maps\Reviews-Estado\\review-Nevada'

In [8]:
def Cartepas_Df(carpeta):
    dataframes = []
    for archivo in os.listdir(carpeta):
        if archivo.endswith('.json'):
            ruta_completa = os.path.join(carpeta, archivo)
            try:
                df = pd.read_json(ruta_completa, lines=True)
                dataframes.append(df)
            except Exception as e:
                print(f"Error al cargar el archivo {archivo}: {e}")
    df_final = pd.concat(dataframes, ignore_index=True)
    return df_final

In [9]:
California = Cartepas_Df(carpeta_json)

In [10]:
Florida = Cartepas_Df(carpeta_json2)

In [11]:
Nevada = Cartepas_Df(carpeta_json2)

In [12]:
print(California.shape)
California.head()

(2700000, 8)


,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.089912e+20,Song Ro,1609909927056,5,Love there korean rice cake.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,1.112903e+20,Rafa Robles,1612849648663,5,Good very good,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
2,1.126404e+20,David Han,1583643882296,4,They make Korean traditional food very properly.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
3,1.174403e+20,Anthony Kim,1551938216355,5,Short ribs are very delicious.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
4,1.005808e+20,Mario Marzouk,1494910901933,5,Great food and prices the portions are large,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49


In [13]:
California.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2700000 entries, 0 to 2699999
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   int64  
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 164.8+ MB


In [14]:
print(California.describe())

            user_id          time        rating
count  2.700000e+06  2.700000e+06  2.700000e+06
mean   1.093547e+20  1.551464e+12  4.313987e+00
std    5.239594e+18  4.648164e+10  1.124260e+00
min    1.000000e+20  6.626016e+11  1.000000e+00
25%    1.048907e+20  1.522967e+12  4.000000e+00
50%    1.093188e+20  1.555199e+12  5.000000e+00
75%    1.138802e+20  1.582251e+12  5.000000e+00
max    1.184467e+20  1.631149e+12  5.000000e+00


In [15]:
print(California.isnull().sum())

user_id          0
name             0
time             0
rating           0
text       1170964
pics       2600710
resp       2454831
gmap_id          0
dtype: int64


In [16]:
def get_diccionario(lista):
    try:
        return lista[0] if isinstance(lista, list) and len(lista) > 0 else None
    except IndexError:
        return None

In [17]:
def extract_url_from_list(lst):
    if isinstance(lst, list) and len(lst) > 0:
        return lst[0]
    return None

In [18]:
California['pics'] = California['pics'].apply(get_diccionario)

In [19]:
df_pics = pd.json_normalize(California['pics'])

In [20]:
df_pics['url'] = df_pics['url'].apply(extract_url_from_list)

In [21]:
California['pics']= df_pics['url']

In [22]:
df_resp = pd.json_normalize(California['resp'])

In [23]:
California['resp'] = df_resp['text']

In [24]:
California['time'] = pd.to_datetime(California['time'], unit='ms')

In [25]:
California['weekday'] = California['time'].dt.day_name()

In [26]:
California['time'] = California['time'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [27]:
California['time'] = pd.to_datetime(California['time'])

In [28]:
California[['gmap_id1', 'gmap_id2']] = California['gmap_id'].str.split(':', expand=True)

In [29]:
California.drop(columns=['gmap_id',], inplace=True)

In [30]:
California['text'] = California['text'].fillna('')
California['pics'] = California['pics'].fillna('')
California['resp'] = California['resp'].fillna('')

In [31]:
California.head()

,user_id,name,time,rating,text,pics,resp,weekday,gmap_id1,gmap_id2
0,1.089912e+20,Song Ro,2021-01-06 05:12:07,5,Love there korean rice cake.,,,Wednesday,0x80c2c778e3b73d33,0xbdc58662a4a97d49
1,1.112903e+20,Rafa Robles,2021-02-09 05:47:28,5,Good very good,,,Tuesday,0x80c2c778e3b73d33,0xbdc58662a4a97d49
2,1.126404e+20,David Han,2020-03-08 05:04:42,4,They make Korean traditional food very properly.,,,Sunday,0x80c2c778e3b73d33,0xbdc58662a4a97d49
3,1.174403e+20,Anthony Kim,2019-03-07 05:56:56,5,Short ribs are very delicious.,,,Thursday,0x80c2c778e3b73d33,0xbdc58662a4a97d49
4,1.005808e+20,Mario Marzouk,2017-05-16 05:01:41,5,Great food and prices the portions are large,,,Tuesday,0x80c2c778e3b73d33,0xbdc58662a4a97d49


In [32]:
Florida.head()

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.014719e+20,Julie A. Gerber,1628003250740,1,Update: Their “reply” to my review amounted to...,None,"{'time': 1627042799532, 'text': 'Thank you for...",0x8893863ea87bd5dd:0x9383ebf973e74abb
1,1.154772e+20,Martin Sheffield,1595031217005,5,He's a knowledgeable doctor but the way he run...,None,"{'time': 1582464056733, 'text': 'Thank you for...",0x8893863ea87bd5dd:0x9383ebf973e74abb
2,1.018050e+20,Brian Truett,1522924253567,5,"Best doctor I've ever had, I never wait to be ...",None,None,0x8893863ea87bd5dd:0x9383ebf973e74abb
3,1.063444e+20,Tina Sun,1467907819586,1,I was told he is a good doctor. I was trying t...,None,None,0x8893863ea87bd5dd:0x9383ebf973e74abb
4,1.008751e+20,James Haynes,1480683415081,5,Takes the time to actually get to know his pat...,None,None,0x8893863ea87bd5dd:0x9383ebf973e74abb


In [33]:
print(Florida.shape)

(2850000, 8)


In [34]:
print(Florida.describe())

            user_id          time        rating
count  2.850000e+06  2.850000e+06  2.850000e+06
mean   1.093032e+20  1.555140e+12  4.313658e+00
std    5.276713e+18  4.360793e+10  1.170064e+00
min    1.000000e+20  1.041379e+12  1.000000e+00
25%    1.047751e+20  1.527462e+12  4.000000e+00
50%    1.092897e+20  1.557678e+12  5.000000e+00
75%    1.138648e+20  1.585605e+12  5.000000e+00
max    1.184467e+20  1.631205e+12  5.000000e+00


In [35]:
print(Florida.isnull().sum())

user_id          0
name             0
time             0
rating           0
text       1079510
pics       2745810
resp       2394601
gmap_id          0
dtype: int64


In [36]:
Florida['time'] = pd.to_datetime(Florida['time'], unit='ms')

In [37]:
Florida['weekday'] = Florida['time'].dt.day_name()

In [38]:
Florida['time'] = Florida['time'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [39]:
Florida['time'] = pd.to_datetime(Florida['time'])

In [40]:
df_resp_Florida = pd.json_normalize(Florida['resp'])

In [41]:
Florida['resp'] = df_resp_Florida['text']

In [42]:
Florida['pics'] = Florida['pics'].apply(get_diccionario)

In [43]:
df_pics_Florida = pd.json_normalize(Florida['pics'])

In [44]:
df_pics_Florida['url'] = df_pics_Florida['url'].apply(extract_url_from_list)

In [45]:
Florida['pics']= df_pics_Florida['url']

In [46]:
Florida[['gmap_id1', 'gmap_id2']] = Florida['gmap_id'].str.split(':', expand=True)

In [47]:
Florida.drop(columns=['gmap_id',], inplace=True)

In [48]:
Florida['text'] = Florida['text'].fillna('')
Florida['pics'] = Florida['pics'].fillna('')
Florida['resp'] = Florida['resp'].fillna('')

In [49]:
Florida.head()

,user_id,name,time,rating,text,pics,resp,weekday,gmap_id1,gmap_id2
0,1.014719e+20,Julie A. Gerber,2021-08-03 15:07:30,1,Update: Their “reply” to my review amounted to...,,Thank you for the feedback. We would like to w...,Tuesday,0x8893863ea87bd5dd,0x9383ebf973e74abb
1,1.154772e+20,Martin Sheffield,2020-07-18 00:13:37,5,He's a knowledgeable doctor but the way he run...,,Thank you for reaching out. I’m forwarding you...,Saturday,0x8893863ea87bd5dd,0x9383ebf973e74abb
2,1.018050e+20,Brian Truett,2018-04-05 10:30:53,5,"Best doctor I've ever had, I never wait to be ...",,,Thursday,0x8893863ea87bd5dd,0x9383ebf973e74abb
3,1.063444e+20,Tina Sun,2016-07-07 16:10:19,1,I was told he is a good doctor. I was trying t...,,,Thursday,0x8893863ea87bd5dd,0x9383ebf973e74abb
4,1.008751e+20,James Haynes,2016-12-02 12:56:55,5,Takes the time to actually get to know his pat...,,,Friday,0x8893863ea87bd5dd,0x9383ebf973e74abb


In [50]:
Nevada.head()

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.014719e+20,Julie A. Gerber,1628003250740,1,Update: Their “reply” to my review amounted to...,None,"{'time': 1627042799532, 'text': 'Thank you for...",0x8893863ea87bd5dd:0x9383ebf973e74abb
1,1.154772e+20,Martin Sheffield,1595031217005,5,He's a knowledgeable doctor but the way he run...,None,"{'time': 1582464056733, 'text': 'Thank you for...",0x8893863ea87bd5dd:0x9383ebf973e74abb
2,1.018050e+20,Brian Truett,1522924253567,5,"Best doctor I've ever had, I never wait to be ...",None,None,0x8893863ea87bd5dd:0x9383ebf973e74abb
3,1.063444e+20,Tina Sun,1467907819586,1,I was told he is a good doctor. I was trying t...,None,None,0x8893863ea87bd5dd:0x9383ebf973e74abb
4,1.008751e+20,James Haynes,1480683415081,5,Takes the time to actually get to know his pat...,None,None,0x8893863ea87bd5dd:0x9383ebf973e74abb


In [51]:
print(Florida.shape)
print(Florida.describe())
print(Florida.isnull().sum())

(2850000, 10)
            user_id                           time        rating
count  2.850000e+06                        2850000  2.850000e+06
mean   1.093032e+20  2019-04-13 07:13:13.878512384  4.313658e+00
min    1.000000e+20            2003-01-01 00:00:00  1.000000e+00
25%    1.047751e+20            2018-05-27 23:01:02  4.000000e+00
50%    1.092897e+20            2019-05-12 16:14:12  5.000000e+00
75%    1.138648e+20            2020-03-30 21:55:06  5.000000e+00
max    1.184467e+20            2021-09-09 16:22:53  5.000000e+00
std    5.276713e+18                            NaN  1.170064e+00
user_id     0
name        0
time        0
rating      0
text        0
pics        0
resp        0
weekday     0
gmap_id1    0
gmap_id2    0
dtype: int64


In [52]:
Nevada['time'] = pd.to_datetime(Nevada['time'], unit='ms')
Nevada['weekday'] = Nevada['time'].dt.day_name()
Nevada['time'] = Nevada['time'].dt.strftime('%Y-%m-%d %H:%M:%S')
Nevada['time'] = pd.to_datetime(Nevada['time'])

In [53]:
df_resp_Nevada = pd.json_normalize(Nevada['resp'])
Nevada['resp'] = df_resp_Nevada['text']

In [77]:
Nevada['pics'] = Nevada['pics'].apply(get_diccionario)

In [55]:
df_pics_Nevada = pd.json_normalize(Nevada['pics'])

In [56]:
df_pics_Nevada['url'] = df_pics_Nevada['url'].apply(extract_url_from_list)

In [57]:
Nevada['pics']= df_pics_Nevada['url']

In [58]:
Nevada[['gmap_id1', 'gmap_id2']] = Nevada['gmap_id'].str.split(':', expand=True)
Nevada.drop(columns=['gmap_id',], inplace=True)

In [59]:
Nevada['text'] = Nevada['text'].fillna('')
Nevada['pics'] = Nevada['pics'].fillna('')
Nevada['resp'] = Nevada['resp'].fillna('')

In [60]:
Nevada.head()

,user_id,name,time,rating,text,pics,resp,weekday,gmap_id1,gmap_id2
0,1.014719e+20,Julie A. Gerber,2021-08-03 15:07:30,1,Update: Their “reply” to my review amounted to...,,Thank you for the feedback. We would like to w...,Tuesday,0x8893863ea87bd5dd,0x9383ebf973e74abb
1,1.154772e+20,Martin Sheffield,2020-07-18 00:13:37,5,He's a knowledgeable doctor but the way he run...,,Thank you for reaching out. I’m forwarding you...,Saturday,0x8893863ea87bd5dd,0x9383ebf973e74abb
2,1.018050e+20,Brian Truett,2018-04-05 10:30:53,5,"Best doctor I've ever had, I never wait to be ...",,,Thursday,0x8893863ea87bd5dd,0x9383ebf973e74abb
3,1.063444e+20,Tina Sun,2016-07-07 16:10:19,1,I was told he is a good doctor. I was trying t...,,,Thursday,0x8893863ea87bd5dd,0x9383ebf973e74abb
4,1.008751e+20,James Haynes,2016-12-02 12:56:55,5,Takes the time to actually get to know his pat...,,,Friday,0x8893863ea87bd5dd,0x9383ebf973e74abb
